In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob, os

%config InlineBackend.figure_format='retina'

In [ ]:
#instrument_list = ['servando', 'belen', 'eva', 'mrallsop', 'alba'] 
instrument_list = ['servando','belen', 'eva', 'voices3',  'voices2'] 

DRIVE_BASE_DIR = '/content/drive/My Drive/SMC 09/DDSP/audio'

fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1, 1, 1)
legend=[]

for INSTRUMENT in instrument_list:

  LOG_DIR = DRIVE_BASE_DIR + '/' + INSTRUMENT + \
              '_checkpoints/summaries/train/'
  print("Analyzing", LOG_DIR)
  if (INSTRUMENT == 'voices2'):
    legend.append('voices2: eva + belen')
  elif (INSTRUMENT == 'voices3'):
    legend.append('voices3: eva + belen + servando')
  else:
    legend.append(INSTRUMENT)

  os.chdir(LOG_DIR)
  logs=[]
  for file in glob.glob("*.v2"):
      logs.append(file)

  losses = {}
  for log_file in logs:
      for e in tf.compat.v1.train.summary_iterator(LOG_DIR+log_file):
          for v in e.summary.value:
              if v.tag == 'losses/total_loss':
                  tensor_array = tf.io.decode_raw(v.tensor.tensor_content, v.tensor.dtype)
                  losses[e.step] = tensor_array[0]
                  if e.step==40000:
                    print(tensor_array[0])
                    
  #dash = ':' if (INSTRUMENT=='alba' or INSTRUMENT=='mrallsop') else '-'
  #width = 1.5
  
  dash = '-' if (INSTRUMENT=='voices3' or INSTRUMENT=='voices2') else ':'
  width = 1.5

  ax.plot(list(losses.keys()), list(losses.values()), dash, linewidth=width)

ax.set_ylabel('total loss')
ax.set_xlabel('steps')
ax.legend(legend)
ax.set_xlim((0,40000))
ax.grid(b=True)
_ = ax.set_ylim((4.5,9))

